In [186]:
import pandas as pd
from google.colab import auth

In [187]:
auth.authenticate_user()

In [188]:
project_id = "quick-weft-442218-s4"

!gcloud config set project {project_id}

Updated property [core/project].


In [189]:
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Updated IAM policy for project [quick-weft-442218-s4].
bindings:
- members:
  - serviceAccount:service-526406018944@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:goog-sc-dynamic-web-app-js-507@quick-weft-442218-s4.iam.gserviceaccount.com
  role: roles/browser
- members:
  - serviceAccount:goog-sc-dynamic-web-app-js-507@quick-weft-442218-s4.iam.gserviceaccount.com
  role: roles/cloudasset.viewer
- members:
  - serviceAccount:526406018944-compute@developer.gserviceaccount.com
  - serviceAccount:526406018944@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-526406018944@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-526406018944@gcp-sa-config.iam.gserviceaccount.com
  role: roles/cloudconfig.serviceAgent
- members:
  - serviceAccount:service-526406018944@gcf-admin-robot.iam.gserviceacco

In [190]:
!gcloud services enable sqladmin.googleapis.com

In [191]:
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pg8000"] SQLAlchemy==2.0.7

In [192]:
INSTANCE_CONNECTION_NAME = "quick-weft-442218-s4:us-central1:mypostgres"
DB_USER = "postgres"
DB_PASS = "root"
DB_NAME = "chicago_business_intelligence"

In [193]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

In [196]:
# Get Taxis Data
with pool.connect() as db_conn:
  results = db_conn.execute(sqlalchemy.text("SELECT id, trip_id, trip_start_timestamp, trip_end_timestamp, pickup_zip_code, dropoff_zip_code FROM taxi_trips")).fetchall()

taxis = pd.DataFrame(columns = ['id', 'trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'pickup_zip_code', 'dropoff_zip_code'])
for row in results:
  taxis.loc[len(taxis)] = row
print(taxis.head().T)

Empty DataFrame
Columns: []
Index: [id, trip_id, trip_start_timestamp, trip_end_timestamp, pickup_zip_code, dropoff_zip_code]


In [198]:
# Get unemployment Data
with pool.connect() as db_conn:
  results = db_conn.execute(sqlalchemy.text("SELECT id, community_area, community_area_name, assault_homicide, below_poverty_level, crowded_housing, dependency, unemployment, per_capita_income FROM community_area_unemployment")).fetchall()

unemp = pd.DataFrame(columns = ['id', 'community_area', 'community_area_name', 'assault_homicide', 'below_poverty_level', 'crowded_housing', 'dependency', 'unemployment', 'per_capita_income'])
for row in results:
  unemp.loc[len(unemp)] = row
print(unemp.head().T)

                               0           1       2               3  \
id                             1           2       3               4   
community_area                 1           2       3               4   
community_area_name  Rogers Park  West Ridge  Uptown  Lincoln Square   
assault_homicide             7.7         5.8     5.4               5   
below_poverty_level         22.7        15.1    22.7             9.5   
crowded_housing              7.9           7     4.6             3.1   
dependency                  28.8        38.3    22.2            25.6   
unemployment                 7.5         7.9     7.7             6.8   
per_capita_income          23714       21375   32355           35503   

                                4  
id                              5  
community_area                  5  
community_area_name  North Center  
assault_homicide                1  
below_poverty_level           7.1  
crowded_housing               0.2  
dependency                 

In [107]:
# Get building permit Data
with pool.connect() as db_conn:
  results = db_conn.execute(sqlalchemy.text("SELECT id, permit_num, permit_type, issue_date, total_fee, reported_cost, community_area FROM building_permits")).fetchall()

build = pd.DataFrame(columns = ['id', 'permit_num', 'permit_type', 'issue_date', 'total_fee', 'reported_cost', 'community_area'])
for row in results:
  build.loc[len(build)] = row
print(build.head().T)

                                          0                             1  \
id                                  2706282                       2713781   
permit_num                        100622550                     100626588   
permit_type     PERMIT - ELEVATOR EQUIPMENT  PERMIT - EASY PERMIT PROCESS   
issue_date              2015-11-05 00:00:00           2015-12-07 00:00:00   
total_fee                              2490                           175   
reported_cost                        666058                          4000   
community_area                           28                            22   

                                           2                             3  \
id                                   2716840                       2708080   
permit_num                         100628184                     100623607   
permit_type     PERMIT - EASY PERMIT PROCESS  PERMIT - EASY PERMIT PROCESS   
issue_date               2015-12-17 00:00:00           2015-11-17 00:00

In [108]:
# Get covid Data
with pool.connect() as db_conn:
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM covid")).fetchall()

covid = pd.DataFrame(columns = ['zip_code', 'week_number', 'week_start', 'week_end', 'cases_weekly', 'cases_cumulative', 'case_rate_weekly', 'case_rate_cumulative', 'percent_tested_positive_weekly', 'percent_tested_positive_cumulative', 'population'])
for row in results:
  covid.loc[len(covid)] = row
print(covid.head().T)

                                                          0  \
zip_code                                              60622   
week_number                                              31   
week_start                          2020-07-26T00:00:00.000   
week_end                            2020-08-01T00:00:00.000   
cases_weekly                                             28   
cases_cumulative                                        877   
case_rate_weekly                                         53   
case_rate_cumulative                                 1661.2   
percent_tested_positive_weekly                        0.029   
percent_tested_positive_cumulative                    0.067   
population                                            52793   

                                                          1  \
zip_code                                              60622   
week_number                                              32   
week_start                          2020-08-02T00:00:0

In [109]:
# Get ccvi Data
with pool.connect() as db_conn:
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM ccvi")).fetchall()

ccvi = pd.DataFrame(columns = ['geography_type', 'community_area_or_zip', 'community_area_name', 'ccvi_score', 'ccvi_category'])
for row in results:
  ccvi.loc[len(ccvi)] = row
print(ccvi.head().T)

                             0      1            2       3       4
geography_type              CA    ZIP           CA     ZIP     ZIP
community_area_or_zip       70  60625            1   60612   60641
community_area_name    Ashburn         Rogers Park                
ccvi_score                45.1   25.5         30.9    31.7    36.1
ccvi_category           MEDIUM    LOW          LOW  MEDIUM  MEDIUM


In [123]:
from google.colab import files
uploaded = files.upload()

Saving all_areas.csv to all_areas (4).csv


In [142]:
from datetime import datetime
timestamp_string = "2023-12-03"
format_string = "%Y-%m-%d"
# REQUIREMENT 2
midway_zip = "60638"
ohare_zip = "60666"

# def get_row_data(row):
#   row = list(row)
#   tripid = row[0]
#   datetime = row[1]
#   month = datetime.month

# def get_month_data()


rowsmid = []
rowsohare = []
# Trips from Midway
with pool.connect() as db_conn:
  resultstaxis = db_conn.execute(sqlalchemy.text("SELECT trip_id, trip_start_timestamp, pickup_zip_code, dropoff_zip_code FROM taxi_trips")).fetchall()
for row in resultstaxis:
  thing = list(row)
  if thing[2] == '60638':
    rowsmid.append(thing)
  elif thing[2] == '60666':
    rowsohare.append(thing)

num_midway = len(rowsmid)
num_ohare = len(rowsohare)

print(rowsmid)
print(rowsohare)

print(num_midway)
print(num_ohare)

[['7de7d6b1667cea33735670f88c50e9631e719f04', datetime.datetime(2023, 12, 31, 23, 45, tzinfo=datetime.timezone.utc), '60638', '60622'], ['408dbbbbb5efe8825b9802e9e47b73bde2cad640', datetime.datetime(2023, 12, 31, 23, 45, tzinfo=datetime.timezone.utc), '60638', '60607'], ['41bb85ba82698b51f96cebd8915b62767fd0698d', datetime.datetime(2023, 12, 31, 23, 45, tzinfo=datetime.timezone.utc), '60638', '60653'], ['d45e012bbd6fffa5ffa8aba12b6d61961c89e9e0', datetime.datetime(2023, 12, 31, 23, 30, tzinfo=datetime.timezone.utc), '60638', '60607'], ['cc3f3f6214a8b4ad9f15f472e0ea734b441728fa', datetime.datetime(2023, 12, 31, 23, 30, tzinfo=datetime.timezone.utc), '60638', '60623'], ['04ef6b2bf7295765e8e848f8526e0bf5271190f7', datetime.datetime(2023, 12, 31, 23, 30, tzinfo=datetime.timezone.utc), '60638', '60657'], ['9f072b4e70e16ebb84b0a2f6ff718150ecc3e345', datetime.datetime(2023, 12, 31, 23, 30, tzinfo=datetime.timezone.utc), '60638', '60640'], ['6668969fdcb4e289f7c099f4042fc8ac31dbfc20', datetime.

In [168]:
middf = pd.DataFrame()
middf["midway_trips"] = [val[3] for val in rowsmid]

middf.to_json()


'{"midway_trips":{"0":"60622","1":"60607","2":"60653","3":"60607","4":"60623","5":"60657","6":"60640","7":"60660","8":"60610","9":"60614","10":"60618","11":"60619","12":"60607","13":"60622","14":"60610","15":"60657","16":"60638","17":"60625","18":"60616","19":"","20":"60652","21":"60638","22":"60616","23":"60622"}}'

In [137]:
# REQ 3
with pool.connect() as db_conn:
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM ccvi")).fetchall()

highzips = []
for row in results:
  if row[4] == 'HIGH':
    if len(row[1]) == 5:
      highzips.append(row[1])
    elif len(row[1]) < 5:
      highzips.append(row[2])

print(highzips)



['Archer Heights', 'Austin', 'West Pullman', '60636', 'Burnside*', 'Brighton Park', 'South Lawndale', 'West Elsdon', '60707', 'New City', '60609', 'North Lawndale', 'Auburn Gresham', 'Chicago Lawn', '60621', '60620', 'West Lawn', 'Montclare', 'Humboldt Park', '60644', '60639', '60628', 'West Englewood', '60651', 'Roseland', 'Belmont Cragin', 'South Deering', 'Englewood', 'Washington Heights', '60623', '60629', 'East Garfield Park', 'Garfield Ridge', 'Fuller Park*', 'Lower West Side', 'Hermosa', '60632', 'Gage Park', '60624']


In [140]:
thezips = ['60634', '60635', "60624", "60639", "60647", "60651", "60634", "60635", "60622", "60624", "60639", "60647", "60651", "60674"]
for i in highzips:
  if i.isnumeric():
    if i not in thezips:
      thezips.append(i)

print(thezips)

['60634', '60635', '60624', '60639', '60647', '60651', '60634', '60635', '60622', '60624', '60639', '60647', '60651', '60674', '60636', '60707', '60609', '60621', '60620', '60644', '60628', '60623', '60629', '60632']


In [145]:
threerows = []
for row in resultstaxis:
  thing = list(row)
  if thing[2] in thezips:
    threerows.append(thing)
  else:
    pass

threedf = pd.DataFrame()
threedf["Zip"] = thezips
threedf["countin"] = 0
threedf["countout"] = 0


for i in threerows:
  threedf.loc[threedf["Zip"] == i[2], "countout"] += 1
  threedf.loc[threedf["Zip"] == i[3], "countin"] += 1

print(threedf)

      Zip  countin  countout
0   60634        0         2
1   60635        0         0
2   60624        1         1
3   60639        1         3
4   60647        4        22
5   60651        1         5
6   60634        0         2
7   60635        0         0
8   60622        7        22
9   60624        1         1
10  60639        1         3
11  60647        4        22
12  60651        1         5
13  60674        0         0
14  60636        0         1
15  60707        0         1
16  60609        1         5
17  60621        3         2
18  60620        1         3
19  60644        5         5
20  60628        6        10
21  60623        4         8
22  60629        4         5
23  60632        0         2


In [151]:
# REQUIREMENT 5
with pool.connect() as db_conn:
  resultspoor = db_conn.execute(sqlalchemy.text("SELECT * FROM community_area_unemployment")).fetchall()

unempl = []
inc = []
comar = []
for row in resultspoor:
  comar.append(row[1])
  unempl.append(row[28])
  inc.append(row[29])

poordf = pd.DataFrame()
poordf["Community Area"] = comar
poordf["Unemployment"] = unempl
poordf["Per Capita Income"] = inc

highunempl = poordf.sort_values(by='Unemployment', ascending=False)
lowincome = poordf.sort_values(by='Per Capita Income')

print(highunempl.head(5))
print(lowincome.head(5))

   Community Area Unemployment Per Capita Income
63             64          9.6             23920
54             55          9.6             22561
20             21          9.3             20489
76             77            9             33364
10             11            9             27280
   Community Area Unemployment Per Capita Income
66             67         34.7             10559
29             30         11.5             10697
25             26         25.2             10951
67             68         21.3             11993
62             63           14             12014


In [160]:
with pool.connect() as db_conn:
  resultsbuild = db_conn.execute(sqlalchemy.text("SELECT permit_num, permit_type, community_area FROM building_permits")).fetchall()
resultsdf = pd.DataFrame()
resultsdf["RESULTS"] = resultsbuild
constrows = []
for row in resultsbuild:
  if row[1] == "PERMIT - NEW CONSTRUCTION":
    constrows.append(row)

print(constrows)
print()

for row in resultsbuild:
  print(row)


[]

('100622550', 'PERMIT - ELEVATOR EQUIPMENT', '28')
('100626588', 'PERMIT - EASY PERMIT PROCESS', '22')
('100628184', 'PERMIT - EASY PERMIT PROCESS', '42')
('100623607', 'PERMIT - EASY PERMIT PROCESS', '35')
('100641877', 'PERMIT - EASY PERMIT PROCESS', '43')
('100652453', 'PERMIT - EASY PERMIT PROCESS', '49')
('100787475', 'PERMIT - EASY PERMIT PROCESS', '42')
('100910640', 'PERMIT - EASY PERMIT PROCESS', '25')
('100924823', 'PERMIT - EASY PERMIT PROCESS', '76')
('100614451', 'PERMIT - ELEVATOR EQUIPMENT', '28')
('100621681', 'PERMIT - EASY PERMIT PROCESS', '76')
('100618681', 'PERMIT - ELEVATOR EQUIPMENT', '28')
('100660147', 'PERMIT - EASY PERMIT PROCESS', '15')
('100622542', 'PERMIT - ELEVATOR EQUIPMENT', '28')
('100606785', 'PERMIT - WRECKING/DEMOLITION', '1')
('100622007', 'PERMIT - EASY PERMIT PROCESS', '6')
('100619558', 'PERMIT - EASY PERMIT PROCESS', '42')
('100605744', 'PERMIT - EASY PERMIT PROCESS', '76')
('100612446', 'PERMIT - EASY PERMIT PROCESS', '76')
('100621758', 

In [162]:
import json

results = {
    "taxis_data" : taxis.head(5).to_json(orient="records"),
    "unemployment_data" : unemp.head(5).to_json(orient="records"),
    "building_permits_data" : build.head(5).to_json(orient="records"),
    "covid_data" : covid.head(5).to_json(orient="records"),
    "ccvi_data" : ccvi.head(5).to_json(orient="records"),
    "Q2" : [{"num_midway" : num_midway},
            {"num_ohare" : num_ohare}],
    "Q3" : threedf.to_json(orient="records"),
    "Q5": [{"high_unemployment" : highunempl.head(5).to_json(orient="records")},
            {"low_income" : lowincome.head(5).to_json(orient="records")}],
    "Q6" : "No results included New Construction Permits"
}


In [165]:
with open('results.json', 'w') as fp:
    json.dump(results, fp)

files.download('results.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>